# Marginal Emissions
## Demonstration on using the marginal emissions feature of NEMED
Using the package <code>nemed</code> to calculate hourly marginal emissions over a historical period.

### Data Preparation
#### Import Packages

In [1]:
import nemed

# To generate plots shown
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# Open plot in browser (optional)
import plotly.io as pio
pio.renderers.default = "browser"

#### Define Parameters
```{tip}
Current Implementation will only load data based on `start_time` day and `end_time` day with the data series commencing
at 04:05:00 and ending at 04:00:00
```

In [3]:
cache = "E:/TEMPCACHE_LX/"
start_time = "2022/07/01 00:00:00" 
end_time = "2022/08/01 00:00:00"
filter_units = None 
filter_regions = None

#### Get data using NEMED

In [4]:
result = nemed.get_marginal_emissions(start_time, end_time, cache, redownload_xml=False)

Retrieving static table Generators and Scheduled Loads.
No existing JSON found for date 2022-06-30 00:00:00
Redownloading XML data
Converting selected 8929 XML files to JSON, of 9505 cached files


100%|██████████| 8929/8929 [00:53<00:00, 167.03it/s]


Reading JSON to pandas Dataframe
Reading selected 8929 JSON files to pandas, of 9505 cached files


100%|██████████| 8929/8929 [01:11<00:00, 124.89it/s]


#### Aggregate/Filter data
Data retrieve from NEMED can be manually filtered (for now) as such...

In [9]:
# Filter select region
results = {}
for region in ['NSW1','QLD1','VIC1','SA1','TAS1']:

    reg_sum = result[result['RegionID'] == region]
    reg_sum = reg_sum.drop_duplicates(subset=['PeriodID','CO2E_EMISSIONS_FACTOR','tech_name'])
    reg_sum = reg_sum.reset_index(drop=True)[['PeriodID','RegionID','Dispatch Type','tech_name','CO2E_EMISSIONS_FACTOR','Date','Hour']]

    # Hourly grouping
    hourly_agg = reg_sum.groupby(by=['Hour']).mean()
    hourly_agg.reset_index(inplace=True)
    results[region] = hourly_agg

An example of the data format this produces for South Australia...

In [11]:
results['SA1']

,Hour,CO2E_EMISSIONS_FACTOR
0,0,0.497064
1,1,0.449482
2,2,0.481346
3,3,0.362853
4,4,0.412288
5,5,0.356128
6,6,0.291657
7,7,0.285021
8,8,0.290229
9,9,0.293776


#### Setup Plotting
##### Marginal Emissions Intensity Plot Format
Showing the average emissions intensity on an hourly basis for the defined study period

In [26]:
def plot_marginal_emissions_intensity(data, color_map={'NSW1':1, 'QLD1':2, 'VIC1':3, 'SA1':4, 'TAS1':5}):

    fig = go.Figure()
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"NEM Regions Mean Marginal Emissions Intensity - July 2022<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.1})
    fig.update_yaxes(title_text="Mean Emissions Intensity (tCO2-e/MWh)")
    fig.update_xaxes(title_text="Hour")

    for region in results.keys():
        fig.add_trace(go.Scatter(x=results[region]['Hour'],y=results[region]['CO2E_EMISSIONS_FACTOR'],\
            name=f"{region}",mode="lines+markers",line_color=colors[color_map[region]]))

    fig.show()

In [27]:
plot_marginal_emissions_intensity(results)

```{include} charts_ex2/hour_agg_allregions.html
```